In [ ]:
import gymnasium as gym
from gymnasium import spaces
from gymnasium.spaces import MultiDiscrete, Discrete

In [ ]:
print(gym.__version__)

In [ ]:
import random 

# Creating a Custom Environment



In [ ]:
# make an environment

class SchedulerEnvironment(gym.Env):
    def __init__(self, scenario_one = True, terminate_num = 200):
        self.scenario_one = scenario_one

        self.queue_one_incoming_packet = 0.0
        self.queue_two_incoming_packet = 0.0
        self.queue_best_effort_incoming_packet = 0.0 

        self.step_counter = 0 # i.e. timeslots
        self.terminate_num = terminate_num

        PRIORITY_ONE_SIZE = 10
        PRIORITY_TWO_SIZE = 10
        BEST_EFFORT_SIZE = 10
        
        # can adjust size of max delay in timeslots after initial observations
        MAX_DELAY = 100

        self.observation_space = spaces.Dict( 
            {
                
                "queue_size": MultiDiscrete([
                    PRIORITY_ONE_SIZE,
                    PRIORITY_TWO_SIZE,
                    BEST_EFFORT_SIZE
                ]),
                
                # number of timeslots since scheduler picked packet from queue
                "max_delay": MultiDiscrete([
                    MAX_DELAY,
                    MAX_DELAY,
                    MAX_DELAY
                ]),
                
#                 boolean for timeslot delay during switch 
#                 zero for no delay, one for delay. 
                "switchCounter": Discrete(2)
                
            }
        )

        self.queue_one_delay = [] 

        self.queue_two_delay = []

        self.queue_best_effort_delay = [] 

        self.queue_delays = dict(
            {
                0: self.queue_one_delay,
                1: self.queue_two_delay,
                2: self.queue_best_effort_delay
            }
        )

        self.queue_one_removed = 0
        self.queue_two_removed = 0
        self.queue_best_effort_removed = 0

        self.removed_all_queues = dict(
            {
                0: self.queue_one_removed,
                1: self.queue_two_removed,
                2: self.queue_best_effort_removed
            }
        )

        self.current_state = None
        
        self.action_space = spaces.Discrete(3)
        
        self._action_to_queue = {
                0: self.observation_space["queue_size"][0],
                1: self.observation_space["queue_size"][1],
                2: self.observation_space["queue_size"][2]
            }

#         assert render_mode is None or render_mode in self.metadata["render_modes"]
#         self.render_mode = render_mode
        
#         """
#         If human-rendering is used, `self.window` will be a reference
#         to the window that we draw to. `self.clock` will be a clock that is used
#         to ensure that the environment is rendered at the correct framerate in
#         human-mode. They will remain `None` until human-mode is used for the
#         first time.
#         """
#         self.window = None
#         self.clock = None

    def _increment_delay(self):
        for i in range(len(self.queue_delays)):    
            for j in range(len(self.queue_delays[i])):
                self.queue_delays[i][j] += 1

            
    # def _add_delay_to_queue(self):
    #     queues = self.observation_space["queue_delay"]
    #     # this will give the index of the queue that had the packet selected
    #     selected_queue = self.action_space.sample() 
    #     for q in range(len(queues)):
    #         if (selected_queue != q):
    #             queues[q] += 1
    #         else:
    #             queues[q] = 0
                

    def _add_packets(self):
        
        PACKET_ARRIVAL_ONE = 0.3
        PACKET_ARRIVAL_TWO = 0.25
        PACKET_ARRIVAL_BEST_EFFORT = 0.4
        
        self.queue_one_incoming_packet += PACKET_ARRIVAL_ONE
        if self.queue_one_incoming_packet >= 1.0:
            self.queue_one_incoming_packet -= 1.0
            self.queue_delays[0].append(0)
                
        self.queue_two_incoming_packet += PACKET_ARRIVAL_TWO
        if self.queue_two_incoming_packet >= 1.0:
            self.queue_two_incoming_packet -= 1.0
            self.queue_delays[1].append(0)


        self.queue_best_effort_incoming_packet += PACKET_ARRIVAL_BEST_EFFORT
        if self.queue_best_effort_incoming_packet >= 1.0:
            self.queue_best_effort_incoming_packet -= 1.0
            self.queue_delays[2].append(0)


        return
     
    def _modify_states(self):

        # will not calculate delay for freshly added packets. 
        self._increment_delay()
        self._add_packets()
        return 
    
    def _get_obs(self):
        return { 
            "queues": self.queue_delays
            }
    
    def _calculate_avg_delay(self, queue):
        queue_size = len(queue)
        if (queue_size == 0):
            return 0
        
        # sum
        sum = 0
        for i in range(queue_size):
            sum += queue[i]
        
        # avg
        avg = sum / queue_size
        return avg

    def _get_info(self):

        avg_one = self._calculate_avg_delay(self.queue_delays[0])
        avg_two = self._calculate_avg_delay(self.queue_delays[1])
        avg_best = self._calculate_avg_delay(self.queue_delays[2])

        # return average delays for each queue. 
        avg_mean_all_queues = [avg_one, avg_two, avg_best]
        
        # return the number of packets removed in each queue. 
        
        packets_removed = [self.removed_all_queues[0], self.removed_all_queues[1], self.removed_all_queues[2]]

        # return the queue num.
        return [avg_mean_all_queues, packets_removed, self.step_counter + 1]
 
    def _initialise_delays(self, queue, size):
        for i in range(size):
            queue.append(0)

    def _set_all_zero(self):
        for i in range((3)):
            self.queue_delays[i].clear()
            self.removed_all_queues[i] = 0

        self.step_counter = 0
        self.current_state = None
        self.queue_one_incoming_packet = 0.0
        self.queue_two_incoming_packet = 0.0
        self.queue_best_effort_incoming_packet = 0.0 
    

    def reset(self, seed=None, options=None):
        # We need the following line to seed self.np_random
        super().reset(seed=seed)

        # reset everything 
        self._set_all_zero()

        # TODO: randomise queues 
        INITIAL_QUEUE_ONE_SIZE = 4
        INITIAL_QUEUE_TWO_SIZE = 5
        INITIAL_QUEUE_BEST_EFFORT_SIZE = 6
        
        self._initialise_delays(self.queue_delays[0], INITIAL_QUEUE_ONE_SIZE)
        self._initialise_delays(self.queue_delays[1], INITIAL_QUEUE_TWO_SIZE)
        self._initialise_delays(self.queue_delays[2], INITIAL_QUEUE_BEST_EFFORT_SIZE)

        self.current_state = None

        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, info
    
    def _reward_function(self, action, packet_delay):
        reward = 0
        MEAN_DELAY_ONE = 6
        MEAN_DELAY_TWO = 4
        # different ways of determining reward for best effort queue
        MEAN_DELAY_BEST_EFFORT = 8

        mean_delays = dict(
            {
                0: MEAN_DELAY_ONE,
                1: MEAN_DELAY_TWO,
                2: MEAN_DELAY_BEST_EFFORT,  
            }
        )
        
        MARGIN_OF_DELAY = 1

        # determine reward (with packet_delay)
#         (mean_delays[action] > packet_delay + MARGIN_OF_DELAY)
        if (action == 2):
        
            # reward this action when mean delays are adjusted
            # for priority queues.
            avg_first_queue = self._calculate_avg_delay(self.queue_delays[0])
            within_margin_first_queue = ( avg_first_queue < mean_delays[0] + MARGIN_OF_DELAY) and ( avg_first_queue > mean_delays[0] - MARGIN_OF_DELAY )
            
            avg_second_queue = self._calculate_avg_delay(self.queue_delays[1])
            within_margin_second_queue = ( avg_second_queue < mean_delays[1] + MARGIN_OF_DELAY) and ( avg_second_queue > mean_delays[1] - MARGIN_OF_DELAY )
            
            if (within_margin_first_queue and within_margin_second_queue):
                reward = 1

            # Punish for not prioritising. 
            else:
                reward = -1

        else:

            # packet delay > mean delay by the margin
            if (mean_delays[action] + MARGIN_OF_DELAY < packet_delay):
                # encourage minimising the delay for this queue.
                reward = 0

            # packet delay < mean delay by the margin
            elif (mean_delays[action] - MARGIN_OF_DELAY > packet_delay):
                # discourage minimising the delay too much.
                reward = -1

            # packet delay within margin, don't perform again. 
            else:
                reward = -1

        return reward 
    
    def _retrieve_packet(self, action):
        # retrieve the packet. 
        packet_delay = self.queue_delays[action].pop(0)
        self.removed_all_queues[action] += 1


        return packet_delay

    def step(self, action):
        packet_delay = 0
        # reward will be -1.
        if len(self.queue_delays[action]) == 0:
            None

        else:
            if self.scenario_one:
                packet_delay = self._retrieve_packet(action)
    
            else: 
                if (action == self.current_state):
                    packet_delay = self._retrieve_packet(action)


        # perform queue switch given the conditions.
        if not self.scenario_one:
            if (action != self.current_state):
                self.current_state = action
        
        reward = self._reward_function(action, packet_delay)
                
        self._modify_states()

        info = self._get_info()

        terminated = False
        if self.step_counter + 1 == self.terminate_num:
            terminated = True
        self.step_counter += 1

        # observation made after modifying states. 
        observation = self._get_obs()

        if self.render_mode == "human":
            self._render_frame()

        return observation, reward, terminated, False, info

    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()


In [ ]:
env = SchedulerEnvironment(terminate_num = 50) 

In [ ]:
def print_info(info):
    print("----------")
    print("current step =", info[2])
    print("-----")
    print("Average delay of first queue:", info[0][0])
    print("Average delay of second queue:", info[0][1])
    print("Average delay of best-effort queue:", info[0][2])
    print("-----")
    print("Packets removed from first queue:", info[1][0])
    print("Packets removed from second queue:", info[1][1])
    print("Packets removed from best-effort queue:", info[1][2])
    

In [ ]:
def print_observation(observation):
    print("----------")
    print("First queue:", observation["queues"][0])
    print("Second queue:", observation["queues"][1])
    print("Best effort queue:", observation["queues"][2])
    

In [ ]:
env.action_space.sample()

In [ ]:
obs, info = env.reset()
print_observation(obs)
print_info(info)

In [ ]:
terminated = False 
truncated = False
env.reset()
score = 0

while not terminated: 
    action = random.randint(0, 2)
    obs, reward, terminated, truncated, info = env.step(action)
    score += reward
    print_observation(obs)
    print_info(info)
print(score)